<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,Discoloration of skin [blotchy] on the neck
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-7.test.csv",header=None)
test.head()

,0,1
0,3,joints poping and cracking
1,8,Arthritis
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'sodium',
 'cramp',
 'limb',
 'weakness',
 'by',
 'on',
 'stomach',
 '-',
 'depression',
 'back',
 'is',
 'at',
 'patients',
 'cramps',
 'that',
 'as',
 'gas',
 'joint',
 'upper',
 'memory',
 'or',
 'drug',
 'leg',
 'mg',
 'legs',
 'pains',
 'knee',
 'headache',
 'group',
 'from',
 'shoulder',
 'unable',
 'not',
 'arthralgia',
 'study',
 'gastrointestinal',
 'neck',
 'treatment',
 'all',
 'this',
 'excessive',
 'foot',
 'arthritis',
 'after',
 'aches',
 'hip',
 'voltaren',
 'numbness',
 'arthrotec',
 'p',
 'disease',
 'insomnia',
 'nausea',
 'release',
 '/',
 'increased',
 'both',
 'asthenia',
 'walk',
 'bleeding',
 'abdominal',
 'skin',
 'be',
 'swelling',
 'dizziness',
 'an',
 'feeling',
 'like',
 'heart',
 'tire

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.786375,3.365469,0.460714,28:38


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.416244,3.307348,0.465023,28:37


In [25]:
learn.save_encoder('pubmed_ask_fold7_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.922061,3.811406,0.305382,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.912363,3.126465,0.465582,01:21


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.358147,2.638186,0.553191,03:22


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.980154,2.315790,0.595745,03:06
1,1.649378,2.147960,0.624531,03:13


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.534794,1.939026,0.658323,03:14
1,1.342108,1.826470,0.678348,03:09


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.238881,1.680676,0.707134,03:10
1,1.032923,1.615673,0.725907,03:13


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.005757,1.546480,0.737171,03:20
1,0.871484,1.482568,0.757196,03:21


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.808853,1.469498,0.734668,03:21
1,0.727668,1.384624,0.762203,03:25


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.685200,1.387850,0.767209,03:19
1,0.648205,1.375708,0.759700,03:18
2,0.563878,1.301624,0.767209,03:31
3,0.484955,1.322461,0.772215,03:21


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.491347,1.280597,0.785982,03:19
1,0.534115,1.296445,0.783479,03:26
2,0.426680,1.269983,0.783479,03:10
3,0.361138,1.253222,0.788486,03:29


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.365002,1.270393,0.806008,03:09
1,0.421490,1.237135,0.792240,03:22
2,0.367841,1.241783,0.803504,03:28
3,0.313457,1.204352,0.802253,03:17


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.368923,1.230549,0.809762,03:25
1,0.341037,1.216865,0.806008,03:11
2,0.301167,1.204479,0.813517,03:19
3,0.250815,1.204912,0.813517,03:11


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.299573,1.169266,0.801001,03:25
1,0.305737,1.239726,0.806008,03:17
2,0.280701,1.221096,0.816020,03:29
3,0.231975,1.230869,0.812265,03:14


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.218129,1.239689,0.811014,03:39
1,0.291272,1.272780,0.809762,03:23
2,0.260954,1.283600,0.812265,03:17
3,0.220464,1.205389,0.813517,03:16


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.211842,1.193705,0.813517,03:14
1,0.245560,1.237765,0.802253,03:24
2,0.231680,1.189641,0.814768,03:24
3,0.191580,1.222515,0.812265,03:25


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.212207,1.307787,0.812265,03:23
1,0.196210,1.223386,0.823529,03:33


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.204145,1.277367,0.803504,03:13
1,0.207702,1.181089,0.827284,03:26


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.190058,1.265389,0.816020,03:38


In [44]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold7')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[126, 8, 8, 8, 8, 8, 8, 510, 10, 13, 536, 652, 27, 27, 749, 150, 30, 31, 320, 32, 37, 37, 37, 37, 741, 70, 448, 514, 49, 49, 100, 290, 808, 49, 49, 49, 49, 240, 49, 49, 49, 100, 100, 50, 903, 53, 53, 53, 58, 494, 59, 80, 61, 61, 62, 1009, 71, 76, 76, 76, 76, 76, 79, 79, 79, 195, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 624, 80, 80, 80, 290, 268, 88, 268, 137, 91, 91, 92, 92, 94, 94, 198, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 100, 100, 100, 937, 265, 100, 337, 100, 100, 100, 100, 10, 881, 108, 111, 111, 111, 111, 114, 290, 842, 122, 627, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 127, 127, 343, 132, 473, 447, 255, 153, 160, 162, 769, 166, 129, 427, 494, 181, 741, 715, 182, 182, 182, 183, 183, 183, 183, 58, 187, 187, 187, 189, 113, 189, 289, 190, 190, 190, 190, 190, 190, 190, 190, 190, 336, 509, 197,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

604
0.6974595842956121
